In [1]:
#standard
import re

#3rd party
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 50

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

from sklearn.model_selection import train_test_split

import miceforest as mf

import catboost

SEED = 1998
np.random.seed(SEED)

In [2]:
data = pd.read_csv("../../../BHL_AI/data/final_train.csv", index_col=0)
# print(len(data))
# data = data.dropna()
print(len(data))

11086


In [3]:
X, y = data.iloc[:,1:], data.iloc[:,0]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
cat_features = ["subject"]

In [6]:
classifier = catboost.CatBoostClassifier(task_type="GPU", devices="0", loss_function='MultiClass')

In [7]:
classifier.fit(X_train, y_train, cat_features=cat_features, verbose=1)

Learning rate set to 0.101307


0:	learn: 1.5670877	total: 42.4ms	remaining: 42.3s
1:	learn: 1.3004237	total: 94.5ms	remaining: 47.2s
2:	learn: 1.1141338	total: 134ms	remaining: 44.5s
3:	learn: 0.9780592	total: 175ms	remaining: 43.7s
4:	learn: 0.8698692	total: 218ms	remaining: 43.4s
5:	learn: 0.7812441	total: 257ms	remaining: 42.6s
6:	learn: 0.7091390	total: 298ms	remaining: 42.2s
7:	learn: 0.6480713	total: 337ms	remaining: 41.8s
8:	learn: 0.5965200	total: 377ms	remaining: 41.5s
9:	learn: 0.5498252	total: 415ms	remaining: 41.1s
10:	learn: 0.5080967	total: 457ms	remaining: 41.1s
11:	learn: 0.4707639	total: 495ms	remaining: 40.8s
12:	learn: 0.4384676	total: 534ms	remaining: 40.5s
13:	learn: 0.4090131	total: 574ms	remaining: 40.4s
14:	learn: 0.3832785	total: 612ms	remaining: 40.2s
15:	learn: 0.3600006	total: 649ms	remaining: 39.9s
16:	learn: 0.3388404	total: 693ms	remaining: 40.1s
17:	learn: 0.3204649	total: 732ms	remaining: 40s
18:	learn: 0.3034649	total: 770ms	remaining: 39.8s
19:	learn: 0.2870047	total: 808ms	remaini

In [8]:
y_hat = classifier.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score

In [10]:
accuracy_score(y_test, y_hat)

0.9941388638412985

# TODO
1. weryfikacja modelu: cross validation
2. metryki: precision, recall, f-1, accuracy, ROC, AUC, AUROC
3. optymalizacja hiperparametrów: random search, grid search

## metrics

In [35]:
from sklearn.model_selection import cross_val_score

In [36]:
def perform_cross_validation(classifier, X_train, y_train, cv=5):
    scores = cross_val_score(classifier, X_train, y_train, cv=cv)
    return scores

In [37]:
cv_scores = perform_cross_validation(classifier=classifier, X_train=X_train, y_train=y_train, cv=5)

cv_scores

Learning rate set to 0.096798
0:	learn: 1.4461780	total: 16.6ms	remaining: 16.6s
1:	learn: 1.2279336	total: 29.1ms	remaining: 14.5s
2:	learn: 1.0703519	total: 41.8ms	remaining: 13.9s
3:	learn: 0.9506295	total: 54.5ms	remaining: 13.6s
4:	learn: 0.8534445	total: 67.8ms	remaining: 13.5s
5:	learn: 0.7733212	total: 80.5ms	remaining: 13.3s
6:	learn: 0.7027636	total: 93.3ms	remaining: 13.2s
7:	learn: 0.6476459	total: 106ms	remaining: 13.1s
8:	learn: 0.5971902	total: 119ms	remaining: 13.1s
9:	learn: 0.5544649	total: 132ms	remaining: 13s
10:	learn: 0.5133661	total: 145ms	remaining: 13.1s
11:	learn: 0.4780752	total: 159ms	remaining: 13.1s
12:	learn: 0.4473773	total: 173ms	remaining: 13.1s
13:	learn: 0.4193018	total: 187ms	remaining: 13.2s
14:	learn: 0.3934330	total: 200ms	remaining: 13.2s
15:	learn: 0.3698136	total: 212ms	remaining: 13.1s
16:	learn: 0.3485925	total: 224ms	remaining: 12.9s
17:	learn: 0.3296755	total: 234ms	remaining: 12.8s
18:	learn: 0.3129498	total: 245ms	remaining: 12.6s
19:	le

array([0.99154453, 0.98928974, 0.99323563, 0.98871968, 0.98871968])

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [27]:
labels = list(data.Activity.unique())

In [28]:
def get_model_score_metrics(labels, y_test, y_pred):
    # accuracy for whole validation set 
    model_accuracy = accuracy_score(y_test, y_pred)
    # metrics indicating model performance for particural classes
    pr_rec_f_supp = precision_recall_fscore_support(y_test, y_pred)
    df_report = pd.DataFrame({'Precision':list(pr_rec_f_supp[0]),
                              'Recall':list(pr_rec_f_supp[1]),
                              'F-Score':list(pr_rec_f_supp[2]),
                              'Support':list(pr_rec_f_supp[3])}, index=labels)
    return model_accuracy, df_report

In [29]:
acc, performance_per_class = get_model_score_metrics(labels=labels, y_test=y_test, y_pred=y_hat)

In [30]:
acc

0.9941388638412985

In [31]:
performance_per_class

,Precision,Recall,F-Score,Support
STANDING,1.000000,1.000000,1.000000,439
WALKING_UPSTAIRS,0.985185,0.992537,0.988848,402
LAYING,0.991870,0.983871,0.987854,372
WALKING,0.994805,0.997396,0.996099,384
SITTING,0.993103,1.000000,0.996540,288
WALKING_DOWNSTAIRS,1.000000,0.990991,0.995475,333


ROC curve -> Wykres True Positive Rate od False Positive Rate